In [22]:
from keras.applications.inception_v3 import InceptionV3,preprocess_input,decode_predictions
from keras.preprocessing import image
import numpy as np
from keras.layers import Dense, GlobalAveragePooling2D,Dropout,Input
from keras.layers.advanced_activations import LeakyReLU, ELU
import tensorflow as tf
import tensorflow as tf
import keras
import os
from keras.layers.core import Activation, Dense, Lambda
import scipy.ndimage
from  scipy import ndimage
from keras.models import Sequential,Model
from keras import backend as K
from IPython.display import display
from keras.models import Model
from keras.models import load_model

In [13]:
base_model  = InceptionV3(weights = 'imagenet', include_top=False)
print('loaded model')

loaded model


In [14]:
#data_gen_args = dict(preprocessing_function=preprocess_input, #Define the dictionary for Image data Generator
  #  rotation_range=30,
  #  width_shift_range=0.2,
  #  height_shift_range=0.2,
  #  shear_range=0.2,
  #  zoom_range=0.2,
  #  horizontal_flip=True,
   # vertical_flip = True)

#train_datagen = image.ImageDataGenerator(**data_gen_args)
#test_datagen = image.ImageDataGenerator(**data_gen_args)

In [15]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)
batch_size= 16

In [23]:
train_set= ('/home/saheed/notebookProj/breakHis/mkfold_keras_8im/fold2/100') 
                                            
test_set =('/home/saheed/notebookProj/breakHis/mkfold_keras_8im/fold2/100')
      

In [24]:
import numpy
import scipy
import pprint
import scipy

print(scipy.version.version)
print(numpy.version.version)


0.19.0
1.14.5


In [25]:
from keras.layers import Conv2D,MaxPooling2D,Flatten
from keras.engine.topology import Layer
dropout = .1
num_output=8
learning_rate = .001

model = Sequential()
model.add(Conv2D(16, (3, 3), padding='same', input_shape=(512, 512, 3), kernel_initializer='he_normal'))
model.add(keras.layers.normalization.BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(dropout))
    
model.add(Conv2D(16, (3, 3), padding='same', kernel_initializer='he_normal'))
model.add(keras.layers.normalization.BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer='he_normal'))
model.add(keras.layers.normalization.BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(dropout))
    
model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer='he_normal'))
model.add(keras.layers.normalization.BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal'))
model.add(keras.layers.normalization.BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(dropout))
    
model.add(Conv2D(64, (3, 3), padding='same', kernel_initializer='he_normal'))
model.add(keras.layers.normalization.BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(MaxPooling2D(pool_size=(2, 2)))
    
model.add(Flatten())
model.add(Dense(512, kernel_initializer="he_normal"))
model.add(keras.layers.normalization.BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(dropout*5))

model.add(Dense(num_output, kernel_initializer="he_normal"))
model.add(keras.layers.normalization.BatchNormalization())
model.add(Activation('softmax'))

Adam = keras.optimizers.Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss="categorical_crossentropy", optimizer=Adam, metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_201 (Conv2D)          (None, 512, 512, 16)      448       
_________________________________________________________________
batch_normalization_205 (Bat (None, 512, 512, 16)      64        
_________________________________________________________________
activation_205 (Activation)  (None, 512, 512, 16)      0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 512, 512, 16)      0         
_________________________________________________________________
conv2d_202 (Conv2D)          (None, 512, 512, 16)      2320      
_________________________________________________________________
batch_normalization_206 (Bat (None, 512, 512, 16)      64        
_________________________________________________________________
activation_206 (Activation)  (None, 512, 512, 16)      0         
__________

In [26]:
import os
import glob
train_loc = os.path.join(str(train_set), 'train')
test_loc = os.path.join(str(test_set), 'test')
num_train = len(glob.glob(train_loc + '/**/*.png', recursive=True))
num_test = len(glob.glob(test_loc + '/**/*.png', recursive=True))
print('num_train', num_train)
print('num_valid', num_test)

steps_per_epoch = np.floor(num_train/batch_size)
validation_steps = np.floor(num_test/batch_size)
epochs_fist = 50
img_dim = 512
 # Image generators
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.1,
    height_shift_range=.2,
    width_shift_range=.2,
    rotation_range=360,
    horizontal_flip=True,
    vertical_flip=True)

generator = datagen.flow_from_directory(
        train_loc,
        target_size=(img_dim, img_dim),
        batch_size=batch_size,
        class_mode='categorical')

valid_datagen = ImageDataGenerator( rescale=1./255)

valid_generator = datagen.flow_from_directory(
        test_loc,
        target_size=(img_dim, img_dim),
        batch_size=batch_size,
        class_mode='categorical')
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(num_output, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in model.layers:
    layer.trainable = False
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

hist = model.fit_generator(generator,
        validation_data=valid_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=epochs_fist,
        validation_steps=validation_steps)

print('Make------------------')


model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'



num_train 1461
num_valid 620
Found 1461 images belonging to 8 classes.
Found 620 images belonging to 8 classes.
Epoch 1/50
91/91 [==============================] - 3263s 36s/step - loss: 1.9544 - acc: 0.3216 - val_loss: 2.0368 - val_acc: 0.1677
Epoch 2/50
91/91 [==============================] - 3170s 35s/step - loss: 1.9561 - acc: 0.3229 - val_loss: 2.0295 - val_acc: 0.1774
Epoch 3/50
91/91 [==============================] - 3104s 34s/step - loss: 1.9607 - acc: 0.3122 - val_loss: 2.0423 - val_acc: 0.1758
Epoch 4/50
91/91 [==============================] - 3110s 34s/step - loss: 1.9536 - acc: 0.3209 - val_loss: 2.0363 - val_acc: 0.1742
Epoch 5/50
91/91 [==============================] - 3123s 34s/step - loss: 1.9563 - acc: 0.3331 - val_loss: 2.0360 - val_acc: 0.1742
Epoch 6/50
91/91 [==============================] - 3102s 34s/step - loss: 1.9538 - acc: 0.3389 - val_loss: 2.0345 - val_acc: 0.1919
Epoch 7/50
91/91 [==============================] - 3122s 34s/step - loss: 1.9589 - acc: 0